TH Nürnberg - Neural Network Translator - Christoph Brandl, Philipp Grandeit

# PoC: Creation of a frontend for CNTK framework

This jupyter notebook provides an exemplary implementation of the extraction of information from a neural network model created with the CNTK framework.

This notebook needs Python 3.6 in order to install the cntk module, because no cntk module 

In [ ]:
import cntk as C
import numpy as np
from sklearn.preprocessing import scale

# Building the network

A feed-forward network with multiple dense layers is created.

In [ ]:
input_dim = 8
num_output_classes = 1
mysamplesize = 64
num_hidden_layers = 1
hidden_layers_dim = 8

input = C.input_variable(input_dim)
label = C.input_variable(num_output_classes)

In [ ]:
def create_model(features):
    with C.layers.default_options(init=C.layers.glorot_uniform(), activation=C.sigmoid):
        h = features
        for _ in range(num_hidden_layers):
            h = C.layers.Dense(hidden_layers_dim, activation=C.sigmoid)(h)
        last_layer = C.layers.Dense(num_output_classes, activation = None)

        return last_layer(h)

In [ ]:
model = create_model(input)

# Save the network model

The functionality of saving notebooks with the CNTK framework is available.

In [ ]:
model.save('cntk_diabetes.model')

# Load the network model

The functionality for loading notebooks with the CNTK framework is also available

In [ ]:
from cntk.ops.functions import load_model

z = load_model("cntk_diabetes.model")

# Read network structure and information from the loaded model

Now the information saved in the model can be read. First we have a look at the structure of the loaded model containing all needed information.

The information can be accessed for each layer so that it would possible to create a json object with the needed structure for the json intermediate format within the Neural-Network-Translator. This shows that it is possible to generate the format from a model created with the CNTK framework and therefore it being possible to create a frontend plugin processing models from different input framework and translating them into the intermediate format.

In [ ]:
layers = C.logging.graph.depth_first_search(
            model, (lambda x : type(x) == C.Function and x.is_block) , depth = 0)
for layer in layers:
    print(layer.op_name) #Typ des Layers z.B. Dense
    print(layer.is_composite)
    print(layer.arguments) 
    print(layer.constants)
    print(layer.parameters)
    print(layer.attributes)
    print('-----')

Also the weights and biases can be read via the parameters field of each layer.

In [ ]:
parameters = z.parameters
for parameter in parameters:
    print(parameter.name, parameter.shape, "\n", parameter.value) 

### END